# Learning Functional Level Design: Arithmetics

$$\renewcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\renewcommand{\bra}[1]{\left\langle{#1}\right|}$$
In this example we will enrich our toolset by using a quantum adder that adds coherently two numbers, and we will explore further capabilities of the synthesize engine!

Let's start with a simple example to explain what do we mean by coherently adding two quantum numbers. Let the two 1-qubit quantum states we want to add be $\ket{x} = \frac{1}{\sqrt{2}}(\ket{0}+\ket{1})$ and $\ket{y}=\ket{1}$. Then when we say $\ket{x+y}$ we want to get $\ket{x}\ket{y}\ket{x+y}=\frac{1}{\sqrt{2}}(\ket{0}\ket{1}\ket{1}+\ket{1}\ket{1}\ket{2})$.

Now, in our example we will take it one step ahead, and the the two numbers we add will be represented by a superposition of two states of 2 qubits:

$$\renewcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\renewcommand{\bra}[1]{\left\langle{#1}\right|}$$

$$
\ket{a} = \frac{1}{\sqrt{2}}(\ket{0} + \ket{3}) = \frac{1}{\sqrt{2}}(\ket{00} + \ket{11})
$$

$$
\ket{b} = \frac{1}{\sqrt{2}}(\ket{1} + \ket{2}) = \frac{1}{\sqrt{2}}(\ket{01} + \ket{10})
$$

Then, the solution we expect is:

$$\renewcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\renewcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$'\ket{a+b}'= \frac{1}{2}(\ket{1} + \ket{2} + \ket{4} + \ket{5})$$

More accurately, is to say that the unitary our model will represent acts as follows:

$$\renewcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\renewcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$ U(\ket{a}\ket{b}\ket{0}) = \ket{a}\ket{b}\ket{a+b} $$

sucht that:

$$\renewcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\renewcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$ U(\ket{a}\ket{b}\ket{0}) = \frac{1}{2}(\ket{0}\ket{1}\ket{1}+\ket{0}\ket{2}\ket{2}+\ket{3}\ket{1}\ket{4}+\ket{3}\ket{2}\ket{5})$$

## A Final Circuit

In this example we are going to synthesize several different circuits for the same functional model. They will be differ in the number of qubits, the depth, the number of 1- and 2-qubit gates, and which gates are used. However, the structure of all the circuits will be the same.

One of the circuits will be:

![arithmetic.png](https://classiq-docs-images.s3.amazonaws.com/arithmetic.png)

which represents the overall structure we will have: two state preparations, and then the adder.

## How We Build This?

As in the previous example, we will first build our functional model, then we will synthesize it into a circuit (with some surprises in the synthesis part ;)) and then we will execute the circuits to get results.

The four steps for building the functional model are:

1. Defining the functional blocks

2. Defining an high-level functional model that will contain the functional blocks

3. Wiring the blocks within the high-level functional model

4. Defining how to execute the resulting quantum circuit

## 1. Defining The Fuctional Blocks

### State Preparations

The first two building blocks are the state preparations:

![arithmetic](https://classiq-docs-images.s3.amazonaws.com/arithmetic_sp.png)

We first need to define the probability distributions of the two registers. The state $\ket{a}$ is an equal superposition of the computational states $\ket{0}$ and $\ket{3}$, while the state $\ket{b}$ is an equal superposition of the computational states $\ket{1}$ and $\ket{2}$.

Therefore the probability distributions are:

In [ ]:
prob_a = [0.5, 0, 0, 0.5]
prob_b = [0, 0.5, 0.5, 0]

We now import the `StatePreparation` object, and initiate two instances of it with the corresponding probability distributions and the desired upper bound for the error value:

In [ ]:
from classiq.builtin_functions import StatePreparation

sp_a = StatePreparation(
    probabilities=prob_a, error_metric={"KL": {"upper_bound": 0.01}}
)
sp_b = StatePreparation(
    probabilities=prob_b, error_metric={"KL": {"upper_bound": 0.01}}
)

In these two state preparations there is no reason for an error, so we can expect the synthesis engine to result in the desired state perfectly.

### Adder

The second component of the algorithm is the adder:

![arithmetic - adder.png](https://classiq-docs-images.s3.amazonaws.com/arithmetic_adder.png)

For this we need to import the `Adder` object and initiate one instance of it. In addition, the parameters this object receives are what are the two arguments it receives. The arguments to be added will be stored in a quantum register, hence these are received as `RegisterUserInput`:

In [ ]:
from classiq import RegisterUserInput
from classiq.builtin_functions import Adder

adder = Adder(
    left_arg=RegisterUserInput(size=2),
    right_arg=RegisterUserInput(size=2),
)

The size of the `RegisteUserInput` is the number of qubits there will be in the register input. Here we know each register will contain 2 qubits.

## 2. Defining The Model

Here we simply initiate an empty `Model` object:

In [ ]:
from classiq import Model

model = Model()

## 3. Wiring The Blocks Within The Model

First we wire the two `StatePreparations` objects into the model:

In [ ]:
a = model.StatePreparation(params=sp_a)
b = model.StatePreparation(params=sp_b)

The outputs of the wirings ($a,b$) are used as input wires for the next component - the adder.

For the wiring of the adder, we give its params (the actual adder object we initiated earlier), and we need to specify now what are the actual inputs! These are the outputs of the wiring of the state preparations:

In [ ]:
adder_out = model.Adder(
    params=adder, in_wires={"left_arg": a["OUT"], "right_arg": b["OUT"]}
)

The last part of the wiring stage is to set the outputs. The wiring of the adder results a dictionary:

In [ ]:
print(adder_out)

With 3 elements, where the last one is the `sum` - the register where the desired result is stored in. We want to tell our model that this is the register of the `sum` we are interested in. We do so by:

In [ ]:
model.set_outputs(
    {"a": adder_out["left_arg"], "b": adder_out["right_arg"], "sum": adder_out["sum"]}
)

## 4. Defining how to execute the resulting quantum circuit

The current example deals with a simple execution, where one asks for measurments on all circuit's outputs (in the computational basis).

In [ ]:
model.sample()

We can also define now our execution preferences

In [ ]:
from classiq.execution import ExecutionPreferences, IBMBackendPreferences
from classiq.synthesis import set_execution_preferences

backend_preferences = IBMBackendPreferences(
    backend_service_provider="IBM Quantum", backend_name="aer_simulator_statevector"
)

serialized_model = model.get_model()

serialized_model = set_execution_preferences(
    serialized_model,
    execution_preferences=ExecutionPreferences(backend_preferences=backend_preferences),
)

Now our model is completed and we can move on to synthesizing some circuits the implement the functionality of the model!

## Synthesizing The First Circuit

We now want to take our model and synthesize some quantum circuits that implement it. As part of the synthesizing process (generating quantum programs process) several optimization methods are used in order to generate the best quantum circuit according to given constraints. Some of these optimization procedure involve some random processes.

In order to receive the same results repeatedly, as it is common in any numerical method, we want to set the seed of the synthesis engine random number generator. We do so by:

In [ ]:
from classiq.model import Preferences
from classiq.synthesis import set_preferences

seed = 206755496
preferences = Preferences(random_seed=seed)
serialized_model = set_preferences(serialized_model, preferences=preferences)

These preferences are passed to the synthesis engine. Let us also ask the engine to optimize over the depth.

In [ ]:
from classiq.model import Constraints
from classiq.synthesis import set_constraints

constraints = Constraints(optimization_parameter="depth")

serialized_model = set_constraints(serialized_model, constraints=constraints)

In [ ]:
with open("learning_arithmetics.qmod", "w") as f:
    f.write(serialized_model)

We can now call the `synthesize` function to get the quantum program:

In [ ]:
from classiq import synthesize

qprog = synthesize(serialized_model)

Before we look at the circuit, we want to check what is the mapping between our variables to the qubits. That is what registers and what qubits actually store the values of the computation:

In [ ]:
from classiq import GeneratedCircuit

circuit = GeneratedCircuit.from_qprog(qprog)

print(circuit.data.qubit_mapping.logical_outputs)

Now let's view the circuit:

In [ ]:
from classiq import show

show(qprog)

And this is what it looks like:

![arithmetic.png](https://classiq-docs-images.s3.amazonaws.com/arithmetic.png)

#### _Interesting tips:_ Saving and reloading circuits

The platform allows us to save our synthesized circuits and to reload them for visualization and even execution at any point in time -

In [ ]:
circuit.save_results("my_arithmetic_circuit.json")

We can reload our saved circuit through the platform's web application. Just drag and drop "my_arithmetic_circuit.json" to the platforms web application as follows:

![drag%20and%20drop%20circuit%20file.png](https://classiq-docs-images.s3.amazonaws.com/drag_and_drop_circuit_file.png)

Furthermore, the platform's web application even supports "dragging and dropping" QASM files!
A QASM file can easily be produced from the circuit object as follows:

In [ ]:
my_file = open("my_arithmetic_circuit.qasm", "w").write(circuit.qasm)

## Executing The First Circuit With A Simulator

Let's execute the circuit we've received to see what are the results. For that we will call the `execute` function:

In [ ]:
from classiq import execute

results_raw = execute(qprog).result()

In [ ]:
from classiq.execution import ExecutionDetails

results = results_raw[0].value
print(results.counts)

The `results.counts` gives us the raw results of the execution in a dictionary format. However, we are interested in understanding the results. Specifically, to understand the results in terms of our task of adding $\ket{a}$ and $\ket{b}$ coherently:

In [ ]:
output_results = results.counts_of_multiple_outputs(["a", "b", "sum"])
print(output_results)

That is, the `output_results` take the total counts of the results, and already divides it into the relevant results per register of interest. Remember that for the binary representation here the LSB is on the left and the MSB is on the right.

We can create a little function that will convert the results to decimal numbers, and then to plot them to see what results have we got:

In [ ]:
def str2num(str):
    return int(str[::-1], 2)


for tupple in output_results.keys():
    print(str2num(tupple[0]), "+", str2num(tupple[1]), "=", str2num(tupple[2]))

SUCCESS :)

Indeed our quantum algorithm succeeded in calculating all the 4 addition operations simultaneously. That is what we mean by coherent addition!

### Executing our first circuit on the platform's web application

We can also execute our circuit through the web application.
Once we have our circuit loaded to the web application, either by the `show` function (used above) or by uploading it from a file, the execution icon will appear on the sidebar on the left. We can click this icon to go to the execution page. This page contains two sections: Quantum Devices and Execution Management. Let's execute our circuit on the Azur Quantum `Ionq.Simulator` with 1000 shots:

![Executing%20arithmetics%20on%20Ionq.png](https://classiq-docs-images.s3.amazonaws.com/Executing_arithmetics_on_Ionq.png)

Once we have our executions preferences are ready we can push the `Run` button on the bottom right corner.
A progress bar will be presented. When the execution is completed, we can view our results:

![Results%20for%20Ionq%20Arithmetics%20xec%20.png](https://classiq-docs-images.s3.amazonaws.com/Results_for_Ionq_Arithmetics_xec.png)

Should we want to download our results for post processing, we can download our results through the `Export as JSON` button. The results file can easily be parsed through our results object as follows:

In [ ]:
from classiq.execution import ExecutionDetails

my_downloaded_results = ExecutionDetails.parse_file(
    "my_ionq_simulator_arithmetic_circuit_execusion_results.json"
)
print(my_downloaded_results.counts)

## Is this The Minimal Number Of Qubits Required?

Let's check what is the width (i.e. number of qubits) of the circuit we have generated:

In [ ]:
print(
    "circuit width: ",
    circuit.data.width,
    " circuit depth: ",
    circuit.transpiled_circuit.depth,
)

And let's ask the Classiq Platform to generate a new circuit, where now we tell it we want the minimal amount of qubits possible:

In [ ]:
constraints = Constraints(optimization_parameter="width")
serialized_model_optimized_for_width = set_constraints(
    serialized_model, constraints=constraints
)
qprog_optimized_for_width = synthesize(serialized_model_optimized_for_width)

And now we can see that from 8 qubits, the circuit is reduced to 7 qubits:

In [ ]:
circuit_optimized_for_width = GeneratedCircuit.from_qprog(qprog_optimized_for_width)

print(
    "circuit width: ",
    circuit_optimized_for_width.data.width,
    " circuit depth: ",
    circuit_optimized_for_width.transpiled_circuit.depth,
)

Wow! Indeed we see that two circuits that have exactly the same functionality of adding two quantum register can differ in their width and depth! This is highly important when we want to generate the optimal quantum algorithm possible.

Let's see how this algorithm is mapped into real hardware. For that we import the `Analyzer` which helps us analyze the circuit we've got:

In [ ]:
from classiq import Analyzer

analyzer = Analyzer(circuit=circuit_optimized_for_width)
analyzer.get_hardware_comparison_table(["Azure Quantum"])
analyzer.plot_hardware_comparison_table()

![harware_comparisn_table_optimized_depth.png](https://classiq-docs-images.s3.amazonaws.com/harware_comparisn_table_optimized_depth.png)

## Is This The Best Circuit For The Specific Hardware?

Ok, so we have optimized the depth of the circuit, but now if we want to execute on some ion-trapped quantum computer, we want to optimize the circuit for this specific hardware! The circuit that we generated might be the shortest one when ignoring a specific hardware, but when having a specific targeted hardware, its

1. connectivity;
2. number of qubits; and
3. native gate-set,

might cause that a different circuit is more suited for it.

Let's synthesize a new circuit, according to the IonQ hardware, which is optimized to be the shorter possible:

In [ ]:
from classiq.model import Preferences

azure_preferences = Preferences(
    backend_service_provider="Azure Quantum", backend_name="ionq", random_seed=seed
)

serialized_model_optimized_for_ionq = set_preferences(
    serialized_model, preferences=azure_preferences
)
serialized_model_optimized_for_ionq = set_constraints(
    serialized_model_optimized_for_ionq, constraints=constraints
)

qprog_optimized_for_ionq = synthesize(serialized_model_optimized_for_ionq)

circuit_optimized_for_ionq = GeneratedCircuit.from_qprog(qprog_optimized_for_ionq)

And now let's check what have changes from the hardware-agnostic optimized circuit (remembering the MULTI QUBIT GATE COUNT is 26 and the TOTAL GATE COUNT is 68):

In [ ]:
analyzer_circuit_optimized_ionq = Analyzer(circuit=circuit_optimized_for_ionq)
analyzer_circuit_optimized_ionq.get_hardware_comparison_table(["Azure Quantum"])
analyzer_circuit_optimized_ionq.plot_hardware_comparison_table()

![hardware_comparison_table_optimized_ionq.png](https://classiq-docs-images.s3.amazonaws.com/hardware_comparison_table_optimized_ionq.png)

Indeed we received a better circuit which is more suited for IonQ!

We saved $24\%$ in the total gate count and $12\%$ in the 2-qubit gate count. These are significant numbers by their own, and be aware that the resources saved, both in numbers and in percentages, get higher as the circuit is larger!

## Comparing The Optimized Circuits

Let's have a look on the circuits and see what is different. First, the hardware-agnostic optimized circuit:

In [ ]:
show(qprog_optimized_for_width)

![arithmetic%20-%20optimized%20for%20depth.png](https://classiq-docs-images.s3.amazonaws.com/arithmetic_optimized_for_depth.png)

and zooming in on the adder:

![arithmetic%20-%20optimized%20for%20depth%20adder.png](https://classiq-docs-images.s3.amazonaws.com/arithmetic_optimized_for_depth_adder.png)

And now for the IonQ-optimized circuit:

In [ ]:
show(qprog_optimized_for_ionq)

![arithmetic%20-%20optimized%20for%20ionq.png](https://classiq-docs-images.s3.amazonaws.com/arithmetic_optimized_for_ionq.png)

And zooming in on the adder:

![atithmetic%20-%20optimized%20for%20ionq%20adder.png](https://classiq-docs-images.s3.amazonaws.com/atithmetic_optimized_for_ionq_adder.png)

These two adders are implementing the exact same thing, but are completely different! This optimization is at the heart of the Classiq Platform, and it is something that could not be handled manually with larger and larger circuits.

## Executing Via Azure

Let's execute the above circuit via Azure on simulator or hardware. We can set the corresponding execution preferences in our quantum program:

In [ ]:
from classiq.execution import (
    AzureBackendPreferences,
    set_quantum_program_execution_preferences,
)

hardware_preferences = AzureBackendPreferences(
    backend_name="ionq.simulator"
)  # ionq.simulator, ionq.qpu

qprog_optimized_for_ionq = set_quantum_program_execution_preferences(
    qprog_optimized_for_ionq,
    preferences=ExecutionPreferences(backend_preferences=hardware_preferences),
)

Then we can execute our circuit:

In [ ]:
results_raw = execute(qprog_optimized_for_ionq).result()

And the results:

In [ ]:
from classiq.execution import ExecutionDetails

results = results_raw[0].value
print("Run via Azure counts:", results.counts)

## All Code Together

In [ ]:
from classiq import (
    Analyzer,
    GeneratedCircuit,
    Model,
    RegisterUserInput,
    execute,
    show,
    synthesize,
)
from classiq.builtin_functions import Adder, StatePreparation
from classiq.execution import (
    AzureBackendPreferences,
    ExecutionDetails,
    ExecutionPreferences,
    IBMBackendPreferences,
    set_quantum_program_execution_preferences,
)
from classiq.model import Constraints, Preferences
from classiq.synthesis import (
    set_constraints,
    set_execution_preferences,
    set_preferences,
)


# defining function
def str2num(str):
    return int(str[::-1], 2)


# defining probabilities
prob_a = [0.5, 0, 0, 0.5]
prob_b = [0, 0.5, 0.5, 0]

# defining state preparation
sp_a = StatePreparation(
    probabilities=prob_a, error_metric={"KL": {"upper_bound": 0.01}}
)
sp_b = StatePreparation(
    probabilities=prob_b, error_metric={"KL": {"upper_bound": 0.01}}
)

# defining the adder
adder = Adder(
    left_arg=RegisterUserInput(size=2),
    right_arg=RegisterUserInput(size=2),
)

# initiating a model
model = Model()

# wiring state preparations
a = model.StatePreparation(params=sp_a)
b = model.StatePreparation(params=sp_b)

# wiring the adder
adder_out = model.Adder(
    params=adder, in_wires={"left_arg": a["OUT"], "right_arg": b["OUT"]}
)
print(adder_out)

# setting the outputs
model.set_outputs(
    {"a": adder_out["left_arg"], "b": adder_out["right_arg"], "sum": adder_out["sum"]}
)
model.sample()
backend_preferences = IBMBackendPreferences(
    backend_service_provider="IBM Quantum", backend_name="aer_simulator_statevector"
)
serialized_model = model.get_model()
serialized_model = set_execution_preferences(
    serialized_model,
    execution_preferences=ExecutionPreferences(backend_preferences=backend_preferences),
)

# fixing the seed
seed = 206755496
preferences = Preferences(random_seed=seed)
serialized_model = set_preferences(serialized_model, preferences=preferences)


# synthesizing the first circuit
constraints = Constraints(optimization_parameter="depth")
serialized_model = set_constraints(serialized_model, constraints=constraints)
qprog = synthesize(serialized_model)


# printing the output mapping
circuit = GeneratedCircuit.from_qprog(qprog)
print(circuit.data.qubit_mapping.logical_outputs)

# vizualizing the circuit
show(qprog)

# save results
circuit.save_results("my_arithmetic_circuit.json")

# save qasm file
my_file = open("my_arithmetic_circuit.qasm", "w").write(circuit.qasm)


# executing the first circuit with a simulator
from classiq import execute

results_raw = execute(qprog).result()

results = results_raw[0].value
print(results.counts)


# output results
output_results = results.counts_of_multiple_outputs(["a", "b", "sum"])
print(output_results)

# understanding the results
for tupple in output_results.keys():
    print(str2num(tupple[0]), "+", str2num(tupple[1]), "=", str2num(tupple[2]))

# loading results from IDE executor
my_downloaded_results = ExecutionDetails.parse_file(
    "my_ionq_simulator_arithmetic_circuit_execusion_results.json"
)
print(my_downloaded_results.counts)


# motivation for shalower circuit
print(
    "circuit width: ",
    circuit.data.width,
    " circuit depth: ",
    circuit.transpiled_circuit.depth,
)

# synthesizing the second circuit
constraints = Constraints(optimization_parameter="width")
serialized_model_optimized_for_width = set_constraints(
    serialized_model, constraints=constraints
)
qprog_optimized_for_width = synthesize(serialized_model_optimized_for_width)
circuit_optimized_for_width = GeneratedCircuit.from_qprog(qprog_optimized_for_width)
print(
    "circuit width: ",
    circuit_optimized_for_width.data.width,
    " circuit depth: ",
    circuit_optimized_for_width.transpiled_circuit.depth,
)


# analyzing the 2nd circuit
analyzer = Analyzer(circuit=circuit_optimized_for_width)
analyzer.get_hardware_comparison_table(["Azure Quantum"])
analyzer.plot_hardware_comparison_table()

# synthesizing the 3rd circuit
azure_preferences = Preferences(
    backend_service_provider="Azure Quantum", backend_name="ionq", random_seed=seed
)
serialized_model_optimized_for_ionq = set_preferences(
    serialized_model, preferences=azure_preferences
)
serialized_model_optimized_for_ionq = set_constraints(
    serialized_model_optimized_for_ionq, constraints=constraints
)
qprog_optimized_for_ionq = synthesize(serialized_model_optimized_for_ionq)
circuit_optimized_for_ionq = GeneratedCircuit.from_qprog(qprog_optimized_for_ionq)


# analyzing the 3rd circuit
analyzer_circuit_optimized_ionq = Analyzer(circuit=circuit_optimized_for_ionq)
analyzer_circuit_optimized_ionq.get_hardware_comparison_table(["Azure Quantum"])
analyzer_circuit_optimized_ionq.plot_hardware_comparison_table()

# viewing the 2nd and 3rd circuits
show(qprog_optimized_for_width)
show(qprog_optimized_for_ionq)

# Azure execution
hardware_preferences = AzureBackendPreferences(
    backend_name="ionq.simulator"
)  # ionq.simulator, ionq.qpu
qprog_optimized_for_ionq = set_quantum_program_execution_preferences(
    qprog_optimized_for_ionq,
    preferences=ExecutionPreferences(backend_preferences=hardware_preferences),
)
results_raw = execute(qprog_optimized_for_ionq).result()
results = results_raw[0].value
print("Run via Azure counts:", results.counts)